# Cyclic Weight Transfer (CWT) with Client-Controlled Cyclic Workflow

In this example, we will demonstrate the Client-Controlled Cyclic Workflow using the Client API with the CIFAR10 dataset. 
This differs from the [Server-Controlled Cyclic Workflow](../cyclic/cyclic.ipynb), as the server is not involved in communication with sensitive information in the case that is it not trusted.

## Client-Controlled Cyclic Workflow

<img src="figs/cyclic_ccwf.png" alt="cyclic ccwf" width=35% height=35% />

The `CyclicServerController` is responsible for managing the lifecycle of the job, and will assign `cyclic_config` and `cyclic_start` tasks for configuration and to begin the training workflow. The configuration includes picking the starting client, result clients, and defining the cyclic order.

The `CyclicClientController` is responsible for the training logic once `cyclic_start` is sent, and the [Cyclic Workflow](../cyclic/cyclic.ipynb#cyclic_workflow) is algorithmically the same as the server-controlled version. The main difference is transferring the model is now encrypted with secure peer-to-peer messaging, and only the result clients receive the model, rather than the server.

See the [docs](https://nvflare.readthedocs.io/en/main/programming_guide/controllers/client_controlled_workflows.html#cyclic-learning) for more information about the Client-Controlled Cyclic Workflow.

## Converting DL training code to FL training code

We will be using the [Client API FL code](../code/fl/train.py) trainer, which is converted from the original [Training a Classifier](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) example.

See [Converting to FL code using Client API](../sag/sag.ipynb#code) for more details.

## Prepare Data

Make sure the CIFAR10 dataset is downloaded with the following script:


In [ ]:
! python ../data/download.py

## Job API

Let's use the Job API to create a CCWF Job.

We use the `add_cyclic()` function to add our server_config and client_config.

First add the `CyclicServerConfig` for the `CyclicServerController` with our desired parameters.
Here we set the required number of rounds, and also increase the max status report interval to 300 seconds.

Next we add the `CyclicClientConfig` for the `CyclicClientController` that handles all `cyclic_*` tasks and maps the `learn_task_name` to the `train` task handled by the `ScriptRunner` with our `train.py` script. The `PTFileModelPersistor` with the initial `Net()` model and the `FullModelShareableGenerator` are also added as components in the `CyclicClientConfig`.

Let's first copy the required files:

In [ ]:
! cp ../code/fl/train.py train.py
! cp ../code/fl/net.py net.py

Then we can use Job API to easily create a job and run in simulator:

In [ ]:

from net import Net

from nvflare.app_common.ccwf.ccwf_job import CCWFJob, CyclicClientConfig, CyclicServerConfig
from nvflare.app_common.ccwf.comps.simple_model_shareable_generator import SimpleModelShareableGenerator
from nvflare.app_opt.pt.file_model_persistor import PTFileModelPersistor
from nvflare.job_config.script_runner import ScriptRunner

n_clients = 2
num_rounds = 3
train_script = "train.py"

job = CCWFJob(name="cifar10_cyclic")

job.add_cyclic(
    server_config=CyclicServerConfig(num_rounds=num_rounds, max_status_report_interval=300),
    client_config=CyclicClientConfig(
        executor=ScriptRunner(script=train_script),
        persistor=PTFileModelPersistor(model=Net()),
        shareable_generator=SimpleModelShareableGenerator(),
    ),
)

job.export_job("/tmp/nvflare/jobs/job_config")
job.simulator_run("/tmp/nvflare/jobs/workdir", n_clients=n_clients, gpu="0")

Ensure that the `train_script` is set to the Client API FL `train.py` code and the model path for the persistor is set to `net.Net`.

## Run the Job

The previous cell exports the job config and executes the job in NVFlare simulator.

If you want to run in production system, you will need to submit this exported job folder to nvflare system.

Lastly, we have the [swarm](../swarm/swarm.ipynb) example, which covers swarm learning and client-controlled cross-site evaluation workflows.